In [1]:
import os
import torch
import scipy.io
import numpy as np
import pandas as pd

from brain_score import time_series_ridge_cv_cuda

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
design_mat = scipy.io.loadmat("./design_matrix/activation_design_mat_1Hz/v25/design_mat_after_pca.mat")
design_mat["pc_mat_1"].shape

(836, 20)

In [4]:
design_mat_full = [design_mat["pc_mat_1"], np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_2"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_3"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_4"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_5"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_6"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_7"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_8"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_9"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_10"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_11"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_12"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_13"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_14"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_15"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_16"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_17"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_18"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_19"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_20"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_21"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_22"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_23"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_24"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_25"]], axis=1),]


In [ ]:
# Define the folder containing the .mat files
folder_path = "./Data/v3_hbdata/extracted_hbo_data_full/TD"  # Change this to your actual path
# folder_path = "./Data/v3_hbdata/extracted_hbo_data_full/WD"  # Change this to your actual path

# Get list of .mat files
files = [f for f in os.listdir(folder_path) if f.endswith('.mat')]

num_files = len(files)
# num_channels = 20  # Each Y has 20 columns
num_channels = 32  # Each Y has 32 columns full brain
brain_score_Z_matrix = np.zeros((num_files, num_channels, len(design_mat_full)))  # Preallocate matrix for weights

num_files, num_channels, brain_score_Z_matrix.shape

(98, 20, (98, 20, 25))

In [19]:

for i, file_name in enumerate(files):
    # Load the .mat file
    file_path = os.path.join(folder_path, file_name)
    data = scipy.io.loadmat(file_path)
    
    # Extract X and Y
    Y = data['hbodata'][0:836,:] # modified to fit the new size of the design matrix

    # Perform regression for each channel
    for j in range(num_channels):
        word_count = 0
        y_j = Y[:, j]  # Extract each channel
        for design_mat_cell in design_mat_full:
            X = design_mat_cell 
            results = time_series_ridge_cv_cuda(X, y_j, device=device)  # Least squares estimation
            brain_score_Z_matrix[i, j, word_count] = results['mean_z']  # Store weight
            word_count += 1
            
    print(f"{file_name} finished...")
    
    break   # <- stop after first file

# Now, W_matrix is (100, 20) containing all regression coefficients
print("Regression completed. W_matrix shape:", brain_score_Z_matrix.shape)

IndexError: index 20 is out of bounds for axis 1 with size 20

In [ ]:
brain_score_Z_matrix.shape

In [ ]:
scipy.io.savemat("brain_score_result_v2_TD.mat", {"brain_score": brain_score_Z_matrix})
# scipy.io.savemat("brain_score_result_v2_WD.mat", {"brain_score": brain_score_Z_matrix})

In [ ]:
# This is for saving a numpy format
# np.save("brain_score_result_raw.npy", brain_score_Z_matrix)
"""file_path=r"F:\MatlabBackUp\PROJECT_HH_ENGLISH\Pilot_Aanalysis_LLM_fNIRS\Data\v3_hbdata\extracted_hbo_data_full\TD\story_4021.mat"
test=scipy.io.loadmat(file_path)
test=test['hbodata']
test.shape"""
